# Scraper for Direct Job Search
### Scaper for directly searching with keywords and location, variable company and location are used for keyword searching

* Website: glassdoor.com
* Library usage: selenium
`
pip install selenium
`
* Results are stored in joblist_raw.csv under data folder
* Fields of output: 'title', 'company', 'location', 'description', 'salary', 'link'
* Refrence: https://github.com/natmod/glassdoor-scrape

## Instruction
* Edit the keywords used for searching in the next cell
* Need to login for the first time usage.
* Query without specific location will return an exception becasue there is actually no remaining page but the previous page indicates there are more pages. Just ignore the exception and run the next cell to save the content into a csv file.
* There might be a pop-up window for watching the jobs. I tried to perform the 'click' operation to skip it but the id/class variable is randomly generated. Just click it manually for now.

In [ ]:
#company='Computer Vision Scientist'
#company='Computer Vision Engineer'
company='Computer Vision'
location=''

In [29]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
import random
from time import sleep
import random
import unicodecsv as csv
import pandas as pd

In [1]:
#This function will take the webdriver, a job title keyword and a location as input,
#it will perform the search using selenium (location is currently commented for more listed jobs)
def search_jobs(driver, job_title_input, location_input):
    job_title = driver.find_element_by_id("sc.keyword")
    job_title.send_keys(job_title_input)
    location = driver.find_element_by_id("sc.location")
    location.send_keys(Keys.CONTROL + "a")
    location.send_keys(Keys.DELETE)
    sleep(5)
    #location.send_keys(location_input)
    driver.find_element_by_id("HeroSearchButton").click()
    sleep(5)
    return

In [31]:
#This function use selenium to find and parse the target element, xpath or classname is used to locate the web element.
#It will automatically click on next page if available. A text output will shows the round number.
def read_listings(driver, listings, idx, results):
    for listing in listings:
        info = listing.find_element_by_class_name("empLoc")
        company = info.find_element_by_tag_name("div")
        company_name=company
        company_location=company
        try:
            company_name, company_location = company.text.split(' – ')
        except:
            pass
        listing.click()
        sleep(5)
        sleep(random.randint(1,5))
        #check for the pop-up
        
        try:
            driver.findElement(By.xpath("//div[contains(text(),'"+ModalStyle__xBtn+"')]")).click
        except:
            pass
        
        salary=''

        try:
            salary=driver.find_element_by_xpath('//*[@id="HeroHeaderModule"]/div[3]/div[4]/div/span').text
        except:
            pass

        
        link=''
        link =driver.find_element_by_xpath('//*[@id="HeroHeaderModule"]/div[3]/div[2]/div[1]/a').get_attribute('href')

        
        description = driver.find_element_by_class_name("jobDescriptionContent").text
        title = driver.find_element_by_class_name("header").find_element_by_tag_name("h1").text
        results[idx] =  {'title' : title, 'company' : company_name, 'location' : company_location, 'description' : description, 'salary' : salary, 'link' : link}
        idx += 1
    return idx, results


In [32]:
#options = webdriver.ChromeOptions()
#options.add_argument("user-data-dir=selenium") 
#driver = webdriver.Chrome(options=options)
#url = "https://www.glassdoor.com/index.htm"

In [36]:
options = webdriver.ChromeOptions()
options.add_argument("user-data-dir=selenium") 
driver = webdriver.Chrome(options=options)
#url = "https://www.glassdoor.com/Job/jobs.htm?sc.keyword=Computer%20Vision%20Engineer&locT=&locId=0&locKeyword=&srs=RECENT_SEARCHES&jl=3061380403"
url = "https://www.glassdoor.com/index.htm"
driver.get(url)

search_jobs(driver, company, location)

listings = driver.find_elements_by_class_name("jl")
idx = 1
results = {}
#find total number of job postings
job_count = int(driver.find_element_by_class_name("jobsCount").text[:-5].replace(',', ''))
print(job_count)

while True:
    #let user know the scraping has started
    print("starting round")
    #find job listing elements on web page
    listings = driver.find_elements_by_class_name("jl")
    #read through job listings and store index and results
    idx, results = read_listings(driver, listings, idx, results)
    #if the job listing index is higher than the total number of job postings found from the search, finish the search
    if idx > job_count:
        print("end of search, final index: " + str(idx))
        break
    #find "next" button to go to next page of job listings
    try:
        next_btn = driver.find_element_by_class_name("next")
    except:
        print("end of search, final index: " + str(idx))
    #if there is no next button, finish the search
    if len(next_btn.find_elements_by_class_name("disabled ")) != 0:
        print("end of search, final index: " + str(idx))
        break
    #click the next button
    next_btn.click()
    #tell webdriver to wait until it finds the job listing elements on the new page
    WebDriverWait(driver, 100).until(lambda driver: driver.find_elements_by_class_name("jl"))
    #let the user know how many job listings have been scraped
    print("end of round, new index: " + str(idx))

46298
starting round
end of round, new index: 33
starting round
end of round, new index: 65
starting round
end of round, new index: 97
starting round
end of round, new index: 129
starting round
end of round, new index: 161
starting round
end of round, new index: 193
starting round
end of round, new index: 225
starting round
end of round, new index: 257
starting round
end of round, new index: 289
starting round
end of round, new index: 321
starting round
end of round, new index: 353
starting round
end of round, new index: 385
starting round
end of round, new index: 417
starting round
end of round, new index: 449
starting round
end of round, new index: 481
starting round
end of round, new index: 513
starting round
end of round, new index: 544
starting round
end of round, new index: 576
starting round
end of round, new index: 608
starting round
end of round, new index: 640
starting round
end of round, new index: 672
starting round
end of round, new index: 704
starting round
end of round, 

TimeoutException: Message: 


In [37]:
with open('../00-data/joblist_raw.csv', 'wb')as csvfile:
    fieldnames = ['title', 'company', 'location', 'description', 'salary', 'link']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames,quoting=csv.QUOTE_ALL, encoding='utf-8')
    writer.writeheader()
    for key,value in results.items():
        writer.writerow(value)

## Contributions 
-By own: 80%  
-By online resources: 20%  

## Citations
1. https://github.com/natmod/glassdoor-scrape

## License

Copyright 2019 COPYRIGHT Yunan Shao

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.